### 1. Kütüphanelerin Import Edilmesi

In [68]:
import pandas as pd
import numpy as np  
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder , RobustScaler
print("kutuphaneler yuklendi")


kutuphaneler yuklendi


### 2. Kesifsel Veri Analizi (EDA)

In [69]:
df = pd.read_csv("../data/raw/german_credit_data.csv", index_col=0)

print("(satir, sutun) sayisi:", df.shape)
print("ilk 5 satir:")
df.head() 

(satir, sutun) sayisi: (1000, 10)
ilk 5 satir:


,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


In [70]:
print("Veri Tipleri ve Genel Bilgi:")
df.info()
print("\nİstatistiksel Özet:")
df.describe(include='all')

Veri Tipleri ve Genel Bilgi:
<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Age               1000 non-null   int64 
 1   Sex               1000 non-null   object
 2   Job               1000 non-null   int64 
 3   Housing           1000 non-null   object
 4   Saving accounts   817 non-null    object
 5   Checking account  606 non-null    object
 6   Credit amount     1000 non-null   int64 
 7   Duration          1000 non-null   int64 
 8   Purpose           1000 non-null   object
 9   Risk              1000 non-null   object
dtypes: int64(4), object(6)
memory usage: 85.9+ KB

İstatistiksel Özet:


,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
count,1000.000000,1000,1000.000000,1000,817,606,1000.000000,1000.000000,1000,1000
unique,NaN,2,NaN,3,4,3,NaN,NaN,8,2
top,NaN,male,NaN,own,little,little,NaN,NaN,car,good
freq,NaN,690,NaN,713,603,274,NaN,NaN,337,700
mean,35.546000,NaN,1.904000,NaN,NaN,NaN,3271.258000,20.903000,NaN,NaN
std,11.375469,NaN,0.653614,NaN,NaN,NaN,2822.736876,12.058814,NaN,NaN
min,19.000000,NaN,0.000000,NaN,NaN,NaN,250.000000,4.000000,NaN,NaN
25%,27.000000,NaN,2.000000,NaN,NaN,NaN,1365.500000,12.000000,NaN,NaN
50%,33.000000,NaN,2.000000,NaN,NaN,NaN,2319.500000,18.000000,NaN,NaN
75%,42.000000,NaN,2.000000,NaN,NaN,NaN,3972.250000,24.000000,NaN,NaN


In [71]:
# Hedef değişkenin dağılımına bakma
fig = px.histogram(df,
                    x='Risk',
                    title='Hedef Değişken Dağılımı',
                    labels={'Risk': 'Kredi Riski'},
                    color='Risk',
                    color_discrete_map={
                        'Good': 'lightblue',
                        'Bad': '#FF0000'  
                    }
                   )
fig.show()


In [72]:
# Risk'e göre sayısal değişkenlerin dağılımı 
numeric_cols = ['Age', 'Credit amount', 'Duration']

for col in numeric_cols:
    fig = px.box(df, x='Risk', y=col, 
                 color='Risk',
                 title=f'{col} - Risk Kategorilerine Göre Dağılım',
                 color_discrete_map={'good': '#2ecc71', 'bad': '#e74c3c'},
                 height=400)
    fig.show()

In [73]:
# Risk'e göre kategorik değişkenlerin analizi 
categorical_cols = ['Sex', 'Job', 'Housing', 'Purpose']

for cat_col in categorical_cols:
    # Crosstab oluştur
    cross_tab = pd.crosstab(df[cat_col], df['Risk'])
    cross_tab_reset = cross_tab.reset_index()
    
    # Long format'a çevir
    cross_tab_long = cross_tab_reset.melt(id_vars=cat_col, 
                                           var_name='Risk', 
                                           value_name='Sayı')
    
    fig = px.bar(cross_tab_long, 
                 x=cat_col, 
                 y='Sayı', 
                 color='Risk',
                 title=f'{cat_col} - Risk Kategorilerine Göre Dağılım',
                 barmode='stack',
                 color_discrete_map={'good': '#2ecc71', 'bad': '#e74c3c'},
                 height=400)
    fig.show()

#### 2.1 Eksik Veri Analizi 

In [74]:
print("Eksik Değerler:\n", df.isnull().sum())
print("\nEksik Değer Yüzdeleri:\n", (df.isnull().mean() * 100))


Eksik Değerler:
 Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64

Eksik Değer Yüzdeleri:
 Age                  0.0
Sex                  0.0
Job                  0.0
Housing              0.0
Saving accounts     18.3
Checking account    39.4
Credit amount        0.0
Duration             0.0
Purpose              0.0
Risk                 0.0
dtype: float64


In [75]:
# Eksik veri olan kolonların görselleştirmesi 
na_cols = ['Saving accounts', 'Checking account']

for col in na_cols:
    # Geçici olarak NaN'ları göster
    temp_data = df[col].fillna('Missing/NA').value_counts().reset_index()
    temp_data.columns = ['Kategori', 'Sayı']
    
    fig = px.bar(temp_data, 
                 x='Kategori', 
                 y='Sayı',
                 title=f'{col} - Veri Dağılımı',
                 color='Sayı',
                 color_continuous_scale='Viridis')
    fig.show()

#### 2.2 Aykiri Değer Analizi

In [76]:
# IQR yöntemiyle aykırı değer tespiti
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return len(outliers), lower_bound, upper_bound

print("IQR Yöntemiyle Aykırı Değer Analizi:")
print("="*60)
for col in numeric_cols:
    n_outliers, lower, upper = detect_outliers_iqr(df, col)
    print(f"{col}:")
    print(f"  Alt Sınır: {lower:.2f}, Üst Sınır: {upper:.2f}")
    print(f"  Aykırı Değer Sayısı: {n_outliers} ({n_outliers/len(df)*100:.2f}%)")
    print()

IQR Yöntemiyle Aykırı Değer Analizi:
Age:
  Alt Sınır: 4.50, Üst Sınır: 64.50
  Aykırı Değer Sayısı: 23 (2.30%)

Credit amount:
  Alt Sınır: -2544.62, Üst Sınır: 7882.38
  Aykırı Değer Sayısı: 72 (7.20%)

Duration:
  Alt Sınır: -6.00, Üst Sınır: 42.00
  Aykırı Değer Sayısı: 70 (7.00%)



**Yorum** Finansal verilerde aykırı değerler gerçek durumları yansıtabilir (örneğin, çok yüksek kredi tutarları). Bu yüzden aykırı değerleri silmek yerine, ölçekleme sırasında `RobustScaler` kullanacağım.

In [77]:
# Aykiri degerlerin box plot ile görselleştirilmesi
df_melted = df[numeric_cols].melt(var_name='Değişken', value_name='Değer')

fig = px.box(df_melted, 
             y='Değer',
             facet_col='Değişken',
             title='Aykırı Değerlerin Box Plot ile Görselleştirilmesi',
             height=400)
fig.update_yaxes(matches=None)  # Her grafik kendi skalasını kullansın
fig.show()

### 3. VERİ ÖN İŞLEME (PREPROCESSING)

In [78]:
# Orijinal veriyi korumak için kopyasını oluşturuyoruz
df_processed = df.copy()
print(f"Preprocessing için kopyalanan veri boyutu: {df_processed.shape}")

Preprocessing için kopyalanan veri boyutu: (1000, 10)


#### 3.1 Eksik Veri Isleme

In [79]:
# 'NA' string değerlerini ve NaN değerlerini 'No Account' kategorisine çeviriyoruz
df_processed['Saving accounts'] = df_processed['Saving accounts'].replace('NA', 'No Account').fillna('No Account')
df_processed['Checking account'] = df_processed['Checking account'].replace('NA', 'No Account').fillna('No Account')

print("Eksik Veri İşleme Sonrası:")
print("\nSaving accounts:")
print(df_processed['Saving accounts'].value_counts())
print("\nChecking account:")
print(df_processed['Checking account'].value_counts())

Eksik Veri İşleme Sonrası:

Saving accounts:
Saving accounts
little        603
No Account    183
moderate      103
quite rich     63
rich           48
Name: count, dtype: int64

Checking account:
Checking account
No Account    394
little        274
moderate      269
rich           63
Name: count, dtype: int64


#### 3.2 Hedef Değişken (Risk) Encoding

In [80]:
# Risk değişkeni (good=0, bad=1)
df_processed['Risk'] = df_processed['Risk'].map({'good': 0, 'bad': 1})

print("Risk Encoding:")
print(df_processed['Risk'].value_counts())


Risk Encoding:
Risk
0    700
1    300
Name: count, dtype: int64


#### 3.3 Train-Test Split (Data Leakage Onlendi)


In [81]:
# X ve y ayrımı
X = df_processed.drop('Risk', axis=1)
y = df_processed['Risk']

# Train-Test split (%80 train, %20 test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train Set: {X_train.shape[0]} samples ({X_train.shape[0]/len(df_processed)*100:.1f}%)")
print(f"Test Set: {X_test.shape[0]} samples ({X_test.shape[0]/len(df_processed)*100:.1f}%)")
print(f"\nTrain Set Risk Dağılımı:")
print(y_train.value_counts())
print(f"\nTest Set Risk Dağılımı:")
print(y_test.value_counts())

Train Set: 800 samples (80.0%)
Test Set: 200 samples (20.0%)

Train Set Risk Dağılımı:
Risk
0    560
1    240
Name: count, dtype: int64

Test Set Risk Dağılımı:
Risk
0    140
1     60
Name: count, dtype: int64


#### 3.4 Ordinal Encoding 

In [82]:
# Saving accounts ve Checking account sıralı (ordinal) değişkenler
# No Account < little < moderate < quite rich/rich

ordinal_mapping = {
    'Saving accounts': ['No Account', 'little', 'moderate', 'quite rich', 'rich'],
    'Checking account': ['No Account', 'little', 'moderate', 'rich']
}

# Ordinal Encoder oluşturuldu
ordinal_encoder = OrdinalEncoder(categories=[ordinal_mapping['Saving accounts'], 
                                              ordinal_mapping['Checking account']])

X_train[['Saving accounts', 'Checking account']] = ordinal_encoder.fit_transform(
    X_train[['Saving accounts', 'Checking account']]
)

X_test[['Saving accounts', 'Checking account']] = ordinal_encoder.transform(
    X_test[['Saving accounts', 'Checking account']]
)
# ordinal encoding train ve test setlerine uygulandı

print("Ordinal Encoding tamamlandı!")
print("\nSaving accounts encoding:")
for i, cat in enumerate(ordinal_mapping['Saving accounts']):
    print(f"  {cat} -> {i}")
print("\nChecking account encoding:")
for i, cat in enumerate(ordinal_mapping['Checking account']):
    print(f"  {cat} -> {i}")

Ordinal Encoding tamamlandı!

Saving accounts encoding:
  No Account -> 0
  little -> 1
  moderate -> 2
  quite rich -> 3
  rich -> 4

Checking account encoding:
  No Account -> 0
  little -> 1
  moderate -> 2
  rich -> 3


#### 3.5 One-Hot Encoding 

In [83]:
# Kategorik değişkenler için One-Hot Encoding
# drop_first=True ile Dummy Variable Trap'ten kaçınıyorum

categorical_cols_for_ohe = ['Sex', 'Job', 'Housing', 'Purpose']

X_train = pd.get_dummies(X_train, columns=categorical_cols_for_ohe, drop_first=True)

X_test = pd.get_dummies(X_test, columns=categorical_cols_for_ohe, drop_first=True)

# Sütunların aynı olduğundan emin olmak icin
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

print(f"One-Hot Encoding tamamlandı!")
print(f"Train set şekli: {X_train.shape}")
print(f"Test set şekli: {X_test.shape}")
print(f"\nYeni öznitelik sayısı: {X_train.shape[1]}")

One-Hot Encoding tamamlandı!
Train set şekli: (800, 18)
Test set şekli: (200, 18)

Yeni öznitelik sayısı: 18


#### 3.6 Olceklendirme (Scailing)

In [84]:
# Aykırı değerler olduğu için RobustScaler kullanıyorum
scaler = RobustScaler()

# Ölçeklendirilecek sayısal sütunlar
numeric_cols_to_scale = ['Age', 'Credit amount', 'Duration']

# Train seti üzerinde fit ve transform
X_train[numeric_cols_to_scale] = scaler.fit_transform(X_train[numeric_cols_to_scale])

# Test seti üzerinde sadece transform
X_test[numeric_cols_to_scale] = scaler.transform(X_test[numeric_cols_to_scale])

print("RobustScaler ile ölçeklendirme tamamlandı!")
print(f"\nÖlçeklendirilen sütunlar: {numeric_cols_to_scale}")
print(f"\nTrain set istatistikleri:")
print(X_train[numeric_cols_to_scale].describe())

RobustScaler ile ölçeklendirme tamamlandı!

Ölçeklendirilen sütunlar: ['Age', 'Credit amount', 'Duration']

Train set istatistikleri:
              Age  Credit amount    Duration
count  800.000000     800.000000  800.000000
mean     0.165982       0.338214    0.230833
std      0.788951       1.036259    0.984777
min     -1.000000      -0.791085   -1.166667
25%     -0.428571      -0.373643   -0.500000
50%      0.000000       0.000000    0.000000
75%      0.571429       0.626357    0.500000
max      3.000000       5.282171    3.500000


#### 3.7 İşlenmiş Verinin Kaydedilmesi

In [85]:
# Train ve test setlerini kaydet
X_train.to_csv('../data/processed/X_train.csv', index=False)
X_test.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

print("İşlenmiş veri setleri kaydedildi!")
print(f"\nKaydedilen dosyalar:")
print("  - ../data/processed/X_train.csv")
print("  - ../data/processed/X_test.csv")
print("  - ../data/processed/y_train.csv")
print("  - ../data/processed/y_test.csv")

İşlenmiş veri setleri kaydedildi!

Kaydedilen dosyalar:
  - ../data/processed/X_train.csv
  - ../data/processed/X_test.csv
  - ../data/processed/y_train.csv
  - ../data/processed/y_test.csv


### 4. Ozet ve Sonuclar

In [86]:
print("="*70)
print("EDA ve ÖN İŞLEME ÖZET RAPORU")
print("="*70)

print("\n1. VERİ SETİ BİLGİLERİ:")
print(f"   - Toplam Örnek Sayısı: {len(df)}")
print(f"   - Öznitelik Sayısı (Orijinal): {df.shape[1]}")
print(f"   - Öznitelik Sayısı (İşlenmiş): {X_train.shape[1]}")

print("\n2. HEDEF DEĞİŞKEN (RISK) DAĞILIMI:")
print(f"   - Good Risk: {(y==0).sum()} (%{(y==0).sum()/len(y)*100:.1f})")
print(f"   - Bad Risk: {(y==1).sum()} (%{(y==1).sum()/len(y)*100:.1f})")

print("\n3. EKSİK VERİ İŞLEME:")
print(f"   - Saving accounts: 'NA' değerleri 'No Account' olarak işlendi")
print(f"   - Checking account: 'NA' değerleri 'No Account' olarak işlendi")

print("\n4. AYKIRI DEĞER ANALİZİ:")
print(f"   - Credit amount, Age, Duration değişkenlerinde aykırı değer tespit edildi")
print(f"   - Aykırı değerler korundu, RobustScaler ile ölçekleme yapıldı")

print("\n5. ENCODING İŞLEMLERİ:")
print(f"   - Risk: Label Encoding (good=0, bad=1)")
print(f"   - Saving/Checking accounts: Ordinal Encoding")
print(f"   - Sex, Job, Housing, Purpose: One-Hot Encoding (drop_first=True)")

print("\n6. ÖLÇEKLEME:")
print(f"   - Yöntem: RobustScaler")
print(f"   - Ölçeklenen değişkenler: Age, Credit amount, Duration")

print("\n7. TRAIN-TEST SPLIT:")
print(f"   - Train Set: {X_train.shape[0]} samples (%80)")
print(f"   - Test Set: {X_test.shape[0]} samples (%20)")
print(f"   - Stratify: Evet (Risk değişkenine göre)")

print("\n8. FİNAL VERİ SETİ:")
print(f"   - X_train: {X_train.shape}")
print(f"   - X_test: {X_test.shape}")
print(f"   - y_train: {y_train.shape}")
print(f"   - y_test: {y_test.shape}")

print("\n" + "="*70)
print("Veri hazır! Bir sonraki adım: Öznitelik Seçimi (Feature Selection)")
print("="*70)

EDA ve ÖN İŞLEME ÖZET RAPORU

1. VERİ SETİ BİLGİLERİ:
   - Toplam Örnek Sayısı: 1000
   - Öznitelik Sayısı (Orijinal): 10
   - Öznitelik Sayısı (İşlenmiş): 18

2. HEDEF DEĞİŞKEN (RISK) DAĞILIMI:
   - Good Risk: 700 (%70.0)
   - Bad Risk: 300 (%30.0)

3. EKSİK VERİ İŞLEME:
   - Saving accounts: 'NA' değerleri 'No Account' olarak işlendi
   - Checking account: 'NA' değerleri 'No Account' olarak işlendi

4. AYKIRI DEĞER ANALİZİ:
   - Credit amount, Age, Duration değişkenlerinde aykırı değer tespit edildi
   - Aykırı değerler korundu, RobustScaler ile ölçekleme yapıldı

5. ENCODING İŞLEMLERİ:
   - Risk: Label Encoding (good=0, bad=1)
   - Saving/Checking accounts: Ordinal Encoding
   - Sex, Job, Housing, Purpose: One-Hot Encoding (drop_first=True)

6. ÖLÇEKLEME:
   - Yöntem: RobustScaler
   - Ölçeklenen değişkenler: Age, Credit amount, Duration

7. TRAIN-TEST SPLIT:
   - Train Set: 800 samples (%80)
   - Test Set: 200 samples (%20)
   - Stratify: Evet (Risk değişkenine göre)

8. FİNAL VERİ